# XGB Classifier with Wrapper-Based FS

Steven Sison | March 9, 2024

## Description

This document will be used to train a model using the reduced feature set obtain by using the wrapper-based method, forward feature selection. The model will be evaluated in terms of the usual metrics (accuracy, precision, F1-score, recall) as well as the training time. The model will also be stored for future evaluation purposes.

## Training the Model

### Preliminaries

#### 1. Loading the Dataset

In [8]:
import pandas as pd                     # For data transformation
import numpy as numpy                   # For scientific calculations
import seaborn as sns                   # For data visualizations
import matplotlib.pyplot as plt         # For plotting
import plotly.graph_objects as go       # For plotting
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay
from xgboost import XGBClassifier, DMatrix, train
from sklearn.pipeline import Pipeline
import time
from datetime import datetime
import joblib
import os

dataset = pd.read_csv("../../../02_feature-engineering/final-datasets/binary_new_Bacud_unbalanced_lexical.csv")      # Loading the dataset

dataset.head()



,url_type,url_length,url_ip_in_domain,url_domain_entropy,url_is_digits_in_domain,url_query_length,url_number_of_parameters,url_number_of_digits,url_string_entropy,url_is_https,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,16,0,3.169925,0,0,0,0,3.375000,0,...,0,0,0,0,0,0,0,0,0,0
1,0,35,0,2.807355,1,0,0,1,4.079143,0,...,0,0,0,0,0,0,0,0,0,0
2,0,31,0,2.921928,0,0,0,1,3.708093,0,...,0,0,0,0,0,0,0,0,0,0
3,1,88,0,2.896292,0,49,4,7,4.660343,0,...,0,0,1,0,0,0,0,0,0,27
4,1,235,0,3.405822,0,194,3,22,4.980518,0,...,0,0,1,0,0,0,0,0,0,27


In [9]:
x_train, x_test, y_train, y_test = train_test_split(dataset.drop(columns=['url_type']), dataset['url_type'], test_size = 0.2, random_state=42)

In [10]:
valid = pd.read_csv("../../../02_feature-engineering/final-datasets/valid_unbalanced_with_lexical.csv")
valid.head()

y_valid = valid['url_type']
X_valid = valid.drop(columns=['url_type'])

#### 2. Preprocessing (Balancing)

In [11]:
dataset['url_type'].value_counts()

url_type
0    724778
1    380244
Name: count, dtype: int64

#### 3. Removing Unnecessary Features

In [12]:
important_features_wrapper_33 = ['url_length',
 'url_domain_entropy',
 'url_is_digits_in_domain',
 'url_number_of_parameters',
 'url_number_of_digits',
 'url_string_entropy',
 'url_path_length',
 'url_host_length',
 'get_tld',
 'url_domain_len',
 'url_num_subdomain',
 'url_number_of_fragments',
 'url_is_encoded',
 'url_number_of_letters',
 'url_num_periods',
 'url_num_of_hyphens',
 'url_num_underscore',
 'url_num_forward_slash',
 'url_num_semicolon',
 'url_num_mod_sign',
 'has_login_in_string',
 'has_signin_in_string',
 'has_logon_in_string',
 'has_loginasp_in_string',
 'has_exe_in_string',
 'has_viewerphp_in_string',
 'has_getImageasp_in_string',
 'has_paypal_in_string',
 'has_dbsysphp_in_string',
 'has_shopping_in_string',
 'has_php_in_string',
 'has_bin_in_string',
 'has_personal_in_string',
 'url_scheme'
 ]

important_features_wrapper_12 = ['url_domain_entropy', 
                              'url_number_of_parameters', 
                              'url_number_of_digits', 
                              'url_path_length', 
                              'url_host_length', 
                              'get_tld', 
                              'url_domain_len', 
                              'url_num_subdomain', 
                              'url_number_of_letters', 
                              'url_num_periods', 
                              'url_num_of_hyphens', 
                              'url_num_forward_slash', 
                              'url_num_semicolon', 
                              'has_login_in_string', 
                              'has_exe_in_string', 
                              'has_php_in_string', 
                              'url_scheme']

X_test_12 = x_test[important_features_wrapper_12]
X_train_12 = x_train[important_features_wrapper_12]

X_test_33 = x_test[important_features_wrapper_33]
X_train_33 = x_train[important_features_wrapper_33]
X_valid_33 = X_valid[important_features_wrapper_33]

X_test_33.head()

,url_length,url_domain_entropy,url_is_digits_in_domain,url_number_of_parameters,url_number_of_digits,url_string_entropy,url_path_length,url_host_length,get_tld,url_domain_len,...,has_exe_in_string,has_viewerphp_in_string,has_getImageasp_in_string,has_paypal_in_string,has_dbsysphp_in_string,has_shopping_in_string,has_php_in_string,has_bin_in_string,has_personal_in_string,url_scheme
165686,58,1.921928,0,1,14,4.659537,32,0,0,5,...,0,0,0,0,0,0,0,0,0,0
712465,76,2.750000,0,0,10,4.626107,42,26,152,8,...,0,0,0,0,0,0,0,0,0,2
335773,133,2.664498,0,7,5,4.657590,10,21,332,14,...,0,0,0,0,0,0,1,0,0,27
533676,30,3.381580,0,0,0,3.989898,1,21,320,17,...,0,0,0,0,0,0,0,0,0,28
642230,44,2.699514,0,0,0,3.772185,44,0,202,14,...,0,0,0,0,0,0,0,0,0,0


#### Hyper-parameter Tuning

##### 12 Features (Purely Lexical)

In [13]:
import optuna
from sklearn.metrics import mean_squared_error # or any other metric
from sklearn.model_selection import train_test_split

# Define the objective function for Optuna
'''def objective_12(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'binary:hinge',
        'eval_metric': 'error',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_12, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = DMatrix(train_data, label=train_target)
    dvalid = DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    
    # Train the model with early stopping
    model = train(param, dtrain, num_boost_round=100000, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    error = mean_squared_error(valid_target, y_pred, squared=False)
    
    return error

# Create an Optuna study and optimize the objective function
study_12 = optuna.create_study(direction='minimize')
study_12.optimize(objective_12, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params_12 = study_12.best_params
best_error = study_12.best_value
print("Best Hyperparameters (12 Features): ", best_params_12)
print("Best Error (12 Features): ", best_error)'''

'def objective_12(trial):\n    # Define the search space for hyperparameters\n    param = {\n        \'objective\': \'binary:hinge\',\n        \'eval_metric\': \'error\',\n        \'eta\': trial.suggest_float(\'eta\', 0.01, 0.3),\n        \'n_estimators\': 100000, # Fix the boosting round and use early stopping\n        \'max_depth\': trial.suggest_int(\'max_depth\', 3, 10),\n        \'subsample\': trial.suggest_float(\'subsample\', 0.5, 1.0),\n        \'colsample_bytree\': trial.suggest_float(\'colsample_bytree\', 0.5, 1.0),\n        \'gamma\': trial.suggest_float(\'gamma\', 0.0, 10.0),\n        \'min_child_weight\': trial.suggest_float(\'min_child_weight\', 0.1, 10.0),\n        \'lambda\': trial.suggest_float(\'lambda\', 0.1, 10.0),\n        \'alpha\': trial.suggest_float(\'alpha\', 0.0, 10.0),\n    }\n    \n    # Split the data into further training and validation sets (three sets are preferable)\n    train_data, valid_data, train_target, valid_target = train_test_split(X_train_12, 

##### 33 Features (Purely Lexical)

In [14]:
# Define the objective function for Optuna
def objective_33(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'binary:hinge',
        'eval_metric': 'error',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_33, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = DMatrix(train_data, label=train_target)
    dvalid = DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    
    # Train the model with early stopping
    model = train(param, dtrain, num_boost_round=100000, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    error = mean_squared_error(valid_target, y_pred, squared=False)
    
    return error

# Create an Optuna study and optimize the objective function
study_33 = optuna.create_study(direction='minimize')
study_33.optimize(objective_33, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params_33 = study_33.best_params
best_error_33 = study_33.best_value
print("Best Hyperparameters (33 Features): ", best_params_33)
print("Best Error (33 Features): ", best_error_33)

[I 2024-04-26 01:16:31,698] A new study created in memory with name: no-name-6cd7662d-8e25-4a73-945c-4e75d2579754
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [01:16:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-error:0.65653
[1]	validation-error:0.65653
[2]	validation-error:0.65653
[3]	validation-error:0.65653
[4]	validation-error:0.65653
[5]	validation-error:0.65653
[6]	validation-error:0.65653
[7]	validation-error:0.65653
[8]	validation-error:0.65653
[9]	validation-error:0.65653
[10]	validation-error:0.65653
[11]	validation-error:0.65653
[12]	validation-error:0.65653
[13]	validation-error:0.65653
[14]	validation-error:0.65653
[15]	validation-error:0.57858
[16]	validation-error:0.57858
[17]	validation-error:0.40515
[18]	validation-error:0.40515
[19]	validation-error:0.40523
[20]	validation-error:0.40522
[21]	validation-error:0.40539
[22]	validation-error:0.40539
[23]	validation-error:0.32294
[24]	validation-error:0.31730
[25]	validation-error:0.31343
[26]	validation-error:0.24997
[27]	validation-error:0.13874
[28]	validation-error:0.13440
[29]	validation-error:0.13218
[30]	validation-error:0.13219
[31]	validation-error:0.13193
[32]	validation-error:0.13202
[33]	validation-erro

[I 2024-04-26 01:18:33,467] Trial 0 finished with value: 0.21543671247483903 and parameters: {'eta': 0.033812544613259096, 'max_depth': 3, 'subsample': 0.8421265790807189, 'colsample_bytree': 0.9175864813688854, 'gamma': 4.2632232394262175, 'min_child_weight': 2.190254521649091, 'lambda': 5.195942426139649, 'alpha': 1.9580886873529546}. Best is trial 0 with value: 0.21543671247483903.
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [01:18:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-error:0.65653
[1]	validation-error:0.65653
[2]	validation-error:0.20130
[3]	validation-error:0.09412
[4]	validation-error:0.09235
[5]	validation-error:0.09405
[6]	validation-error:0.09435
[7]	validation-error:0.09417
[8]	validation-error:0.09464
[9]	validation-error:0.09280
[10]	validation-error:0.09462
[11]	validation-error:0.09397
[12]	validation-error:0.09399
[13]	validation-error:0.09444
[14]	validation-error:0.09386
[15]	validation-error:0.09388
[16]	validation-error:0.09141
[17]	validation-error:0.08373
[18]	validation-error:0.08196
[19]	validation-error:0.08149
[20]	validation-error:0.08130
[21]	validation-error:0.07986
[22]	validation-error:0.07877
[23]	validation-error:0.07823
[24]	validation-error:0.07822
[25]	validation-error:0.07806
[26]	validation-error:0.07756
[27]	validation-error:0.07743
[28]	validation-error:0.07690
[29]	validation-error:0.07515
[30]	validation-error:0.07390
[31]	validation-error:0.07307
[32]	validation-error:0.07246
[33]	validation-erro

[I 2024-04-26 01:19:29,507] Trial 1 finished with value: 0.20441718353264243 and parameters: {'eta': 0.25424004181925725, 'max_depth': 3, 'subsample': 0.7260381562750227, 'colsample_bytree': 0.9980787928891537, 'gamma': 4.654714412219394, 'min_child_weight': 7.857724750870895, 'lambda': 6.64123741054587, 'alpha': 2.8681551795475824}. Best is trial 1 with value: 0.20441718353264243.
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [01:19:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


...

[0]	validation-error:0.65653
[1]	validation-error:0.39796
[2]	validation-error:0.09956
[3]	validation-error:0.07446
[4]	validation-error:0.07413
[5]	validation-error:0.05909
[6]	validation-error:0.05399
[7]	validation-error:0.05210
[8]	validation-error:0.05077
[9]	validation-error:0.05007
[10]	validation-error:0.04923
[11]	validation-error:0.04883
[12]	validation-error:0.04791
[13]	validation-error:0.04701
[14]	validation-error:0.04637
[15]	validation-error:0.04541
[16]	validation-error:0.04495
[17]	validation-error:0.04434
[18]	validation-error:0.04420
[19]	validation-error:0.04391
[20]	validation-error:0.04356
[21]	validation-error:0.04308
[22]	validation-error:0.04267
[23]	validation-error:0.04240
[24]	validation-error:0.04223
[25]	validation-error:0.04202
[26]	validation-error:0.04183
[27]	validation-error:0.04163
[28]	validation-error:0.04144
[29]	validation-error:0.04123
[30]	validation-error:0.04109
[31]	validation-error:0.04077
[32]	validation-error:0.04061
[33]	validation-erro

[I 2024-04-26 01:58:32,785] Trial 98 finished with value: 0.18200845742130492 and parameters: {'eta': 0.27978982288753484, 'max_depth': 10, 'subsample': 0.7507154079591536, 'colsample_bytree': 0.8480724597535044, 'gamma': 1.4753790543987406, 'min_child_weight': 4.254670960186274, 'lambda': 0.2473012186224628, 'alpha': 8.857094530689022}. Best is trial 95 with value: 0.17811343401234572.
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [01:58:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-error:0.65653


[I 2024-04-26 01:58:34,345] Trial 99 pruned. Trial was pruned at iteration 1.


Best Hyperparameters (33 Features):  {'eta': 0.28136470194504565, 'max_depth': 10, 'subsample': 0.8094809699107595, 'colsample_bytree': 0.8112232986253243, 'gamma': 0.468635873848658, 'min_child_weight': 4.042690742485771, 'lambda': 0.7438752818465018, 'alpha': 9.52919689347772}
Best Error (33 Features):  0.17811343401234572


In [15]:
# print(best_params_12)
print(best_params_33)

{'eta': 0.28136470194504565, 'max_depth': 10, 'subsample': 0.8094809699107595, 'colsample_bytree': 0.8112232986253243, 'gamma': 0.468635873848658, 'min_child_weight': 4.042690742485771, 'lambda': 0.7438752818465018, 'alpha': 9.52919689347772}


#### Model Training

In [16]:
from sklearn.model_selection import KFold
import optuna

# Initialize CV
cv = KFold(n_splits=10, shuffle=True, random_state=1)

# best_params_12['objective'] = 'binary:hinge'
# best_params_12['eval_metric'] = 'error'

best_params_33['objective'] = 'binary:hinge'
best_params_33['eval_metric'] = 'error'

# Convert the data into DMatrix format
# dtrain_12 = DMatrix(X_train_12, label=y_train)
# dvalid_12 = DMatrix(X_test_12, label=y_test)

dtrain_33 = DMatrix(X_train_33, label=y_train)
dvalid_33 = DMatrix(X_test_33, label=y_test)
lexical_valid = DMatrix(X_valid_33, label = y_valid)

# Train the Model
# xgb_classifier_12 = train(best_params_12, dtrain_12, num_boost_round=3000)
# y_pred_12 = xgb_classifier_12.predict(dvalid_12)

# print("Model with 12 Features Done.")

xgb_classifier_33 = train(best_params_33, dtrain_33, num_boost_round=3000)
y_pred_33 = xgb_classifier_33.predict(dvalid_33)
y_pred_valid = xgb_classifier_33.predict(lexical_valid)

print("Model with 33 Features Done.")

Model with 33 Features Done.


#### Evaluation

In [17]:
# Classification Report
print("------------- Test Evaluation -------------")
# print(classification_report(y_test, y_pred_12))
print(classification_report(y_test, y_pred_33))

------------- Test Evaluation -------------
              precision    recall  f1-score   support

           0       0.98      0.98      0.98    144867
           1       0.96      0.95      0.96     76138

    accuracy                           0.97    221005
   macro avg       0.97      0.97      0.97    221005
weighted avg       0.97      0.97      0.97    221005



In [18]:
# Classification Report
print("------------- Practical Evaluation -------------")

print(classification_report(y_valid, y_pred_valid))

------------- Practical Evaluation -------------
              precision    recall  f1-score   support

         0.0       1.00      0.00      0.00      1000
         1.0       0.00      0.00      0.00         0

    accuracy                           0.00      1000
   macro avg       0.50      0.00      0.00      1000
weighted avg       1.00      0.00      0.00      1000



c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [19]:
'''# Confusion Matrix for 12 Features
cm_up = confusion_matrix(y_test, y_pred_12, labels=xgb_classifier_12.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier_12.classes_)
disp.plot()
plt.show()'''

'# Confusion Matrix for 12 Features\ncm_up = confusion_matrix(y_test, y_pred_12, labels=xgb_classifier_12.classes_)\ndisp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier_12.classes_)\ndisp.plot()\nplt.show()'

In [20]:
'''# Cross Validation Score
scores = cross_val_score(XGBClassifier(**params_gbm),
                        X_train, y_train, scoring='accuracy', cv=cv).mean()

print(scores)'''

"# Cross Validation Score\nscores = cross_val_score(XGBClassifier(**params_gbm),\n                        X_train, y_train, scoring='accuracy', cv=cv).mean()\n\nprint(scores)"

In [21]:
# Dumping the model
# joblib.dump(xgb_classifier_12, 'xgb_ffs_12.sav')
# joblib.dump(xgb_classifier_33, 'xgb_ffs_33.sav')

In [22]:
import lexical_generator_12
import lexical_generator_33
import time

'''def xgb_predict_maliciousness_12(url):

    numerical_values = lexical_generator_12.lexical_generator(url)
    # print(numerical_values)
    numerical_values = DMatrix(numerical_values)

    match xgb_classifier_12.predict(numerical_values):
        case 0:
            return "Benign"
        case 1:
            return "Malware"
        case 2:
            return "Phishing"
        case 3:
            return "Defacement"'''
        
def xgb_predict_maliciousness_33(url):

    numerical_values = lexical_generator_33.lexical_generator(url)
    # print(numerical_values)
    numerical_values = DMatrix(numerical_values)

    match xgb_classifier_33.predict(numerical_values):
        case 0:
            return "Benign"
        case 1:
            return "Malware"
        case 2:
            return "Phishing"
        case 3:
            return "Defacement"

url = "www.facebook.com/"
print("Current URL: "+url)

'''print("------------- Wrapper-Based (12 Features) -------------")
for i in range(15):
    start = time.perf_counter()
    prediction = xgb_predict_maliciousness_12(url)
    end = time.perf_counter()
    print("Trial "+str(i))
    print(prediction)
    print(end-start)'''

print("------------- Wrapper-Based (33 Features) -------------")
for i in range(15):
    start = time.perf_counter()
    prediction = xgb_predict_maliciousness_33(url)
    end = time.perf_counter()
    print("Trial "+str(i))
    print(prediction)
    print(end-start)

Current URL: www.facebook.com/
------------- Wrapper-Based (33 Features) -------------
Trial 0
Benign
0.14584130002185702
Trial 1
Benign
0.01974979997612536
Trial 2
Benign
0.01934969995636493
Trial 3
Benign
0.020302099990658462
Trial 4
Benign
0.021452800021506846
Trial 5
Benign
0.021469100029207766
Trial 6
Benign
0.020812499918974936
Trial 7
Benign
0.018818599986843765
Trial 8
Benign
0.01853780006058514
Trial 9
Benign
0.018077500048093498
Trial 10
Benign
0.01872920000459999
Trial 11
Benign
0.019917100085876882
Trial 12
Benign
0.01854959991760552
Trial 13
Benign
0.018855800037272274
Trial 14
Benign
0.018644300056621432
